# Gossip Learning Overview

# Implementation

## Dummy Network Implementation

In [1]:
import time
import threading
import traceback

# Packet class
class Packet:
    def __init__(self, src, dest, data):
        self.src = src
        self.dest = dest
        self.data = data
    def __str__(self):
        return "src=" + self.src + ",dest=" + self.dest + ",data=" + str(self.data)

class DummyNet:
    def __init__(self, address, neighbor_addrs = []):
        self.ip = address
        self.neighbor_addrs = neighbor_addrs
        self.outbox = []
        self.inbox = []
        self.active = True
    def init_network(self, registry):
        # Build neighbors
        self.build_neighbors(registry)
        # Init sender and receiver processes
        self.init_sender()
        self.init_receiver()
    def kill(self):
        self.active = False
    def print_(self, message):
        # Network level print messaging
        output = "NET::[" + str(self.ip) + "]::"
        try:
            output = output + str(message)
        except:
            output = output + "Message not printable."
        print(output)
    def build_neighbors(self, registry):
        # The registry is built as a dictionary with key IP address an entry DummyNet object
        self.neighbors = {}
        for addr in self.neighbor_addrs:
            self.neighbors[addr] = registry[addr]
    def init_sender(self):
        # Start sender service
        threading.Thread(target=self.__send, args=()).start()
        pass
    def init_receiver(self):
        # Start receiver service
        threading.Thread(target=self.__receive, args=()).start()
        pass
    def __send(self):
        # Network layer send function
        while self.active:
            try:
                if len(self.outbox):
                    packet = self.outbox.pop(0)
                    self.neighbors[packet.dest].inbox.append(packet)
                    self.print_("Sent: " + str(packet))
            except Exception as e:
                self.print_("Sending error has occurred.")
                traceback.print_exc()
                
    def __receive(self):
        # Receiving/processing function
        while self.active:
            try:
                if len(self.inbox):
                    packet = self.inbox.pop(0)
                    self.print_("Received: " + str(packet))
            except Exception as e:
                self.print_("Receiving error has occurred.")
                traceback.print_exc()
        pass
    def send(self, payload, address):
        # Application layer send function
        # Create packet
        packet = Packet(self.ip, address, payload)
        # Load packet into outbox
        self.outbox.append(packet)

## Client

In [9]:
# A single record.
class Record:
    def __init__(self, ip, weights, expiry):
        self.ip = ip
        self.weights = weights
        self.expiry = expiry
    def step(self, s=1):
        self.expiry -= 1
        if self.expiry <= 0:
            return True
        else:
            return False

# Holds and manages records and requests.
class GuestBook:
    def __init__(self):
        self.records = {}
    def encounter(self, ip, weights, expiry):
        self.records[ip] = Record(ip, weights, expiry)
    def step(self):
        # Increment another step and reduce expirations across records.
        for ip in self.records.keys():
            # If incremental step results in expiration, remove record.
            if self.records[ip].step():
                expunged = self.records.pop(ip)
                

class Client:
    def __init__(self, netNode=None, ip=None, neighbor_addrs=None):
        if netNode is not None:
            self.net = netNode
        else:
            self.net = DummyNet(ip, neighbor_addrs)
    def transmit(self, payload, target_addr):
        self.net.send(payload, target_addr)

## Execution

In [8]:
import time
from datetime import datetime

# Create a graph
graph = {}
graph["10.0.0.1"] = ["10.0.0.2", "10.0.0.3"]
graph["10.0.0.2"] = ["10.0.0.1", "10.0.0.3"]
graph["10.0.0.3"] = ["10.0.0.1", "10.0.0.2"]

# Create nodes for the virtual network
ipRegistry = {}
for addr in graph.keys():
    newNode = DummyNet(addr, graph[addr])
    ipRegistry[addr] = newNode
# Build network (decentralized)
for addr in graph.keys():
    ipRegistry[addr].init_network(ipRegistry)

# Create clients
clientDict = {}
for ip in ipRegistry.keys():
    clientDict[ip] = Client(netNode=ipRegistry[ip])

clientDict["10.0.0.1"].transmit(str(time.time()), "10.0.0.2")
time.sleep(2.5)
clientDict["10.0.0.2"].transmit(str(time.time()), "10.0.0.3")
time.sleep(2.5)

# Kill all nodes
for addr in graph.keys():
    ipRegistry[addr].kill()


NET::[10.0.0.1]::Sent: src=10.0.0.1,dest=10.0.0.2,data=1598289451.5475302
NET::[10.0.0.2]::Received: src=10.0.0.1,dest=10.0.0.2,data=1598289451.5475302
NET::[10.0.0.2]::Sent: src=10.0.0.2,dest=10.0.0.3,data=1598289454.9727702
NET::[10.0.0.3]::Received: src=10.0.0.2,dest=10.0.0.3,data=1598289454.9727702
